In [1]:
import pandas as pd
import numpy as np
import joblib  # Modeli kaydetmek için kütüphane
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler yüklendi.")

Kütüphaneler yüklendi.


In [2]:
def load_and_preprocess_data(path):
    # 1. Veriyi Yükle
    df = pd.read_csv(path)
    
    # 2. Encoding (Type sütunu)
    type_map = {'L': 0, 'M': 1, 'H': 2}
    df['Type'] = df['Type'].map(type_map)
    
    # 3. Feature Engineering (Mühendislik)
    # Fiziksel formülleri burada uyguluyoruz
    df['Power_W'] = df['Torque [Nm]'] * df['Rotational speed [rpm]'] * (2 * np.pi / 60)
    df['Temp_Diff'] = df['Process temperature [K]'] - df['Air temperature [K]']
    df['Strain'] = df['Torque [Nm]'] * df['Tool wear [min]']
    
    # 4. Gereksiz Sütunları Temizle
    drop_cols = ['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
    df = df.drop(drop_cols, axis=1)
    
    return df

# Fonksiyonu kullanarak veriyi hazırla
df_clean = load_and_preprocess_data('../data/ai4i2020.csv')

# X ve y Ayrımı
X = df_clean.drop('Machine failure', axis=1)
y = df_clean['Machine failure']

print(f"İşlenmiş Veri Boyutu: {X.shape}")
display(X.head())

İşlenmiş Veri Boyutu: (10000, 9)


,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Power_W,Temp_Diff,Strain
0,1,298.1,308.6,1551,42.8,0,6951.590560,10.5,0.0
1,0,298.2,308.7,1408,46.3,3,6826.722724,10.5,138.9
2,0,298.1,308.5,1498,49.4,5,7749.387543,10.4,247.0
3,0,298.2,308.6,1433,39.5,7,5927.504659,10.4,276.5
4,0,298.2,308.7,1408,40.0,9,5897.816608,10.5,360.0


In [4]:
# 1. Train-Test Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. SMOTE ile Dengeleme
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 3. Final Modeli Kur (Optimize edilmiş parametrelerle)
# Not: GridSearch'te n_estimators=200 en iyi çıkmıştı.
final_model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# 4. Modeli Eğitme
print("Final Model Eğitiliyor...")
final_model.fit(X_train_resampled, y_train_resampled)
print("Model Eğitimi Tamamlandı.")

# 5. Son Kez Test Etme
y_pred = final_model.predict(X_test)
print("\n--- Final Model Performansı ---")
print(classification_report(y_test, y_pred))

# 6. MODELİ KAYDETME
# Modeli 'models' klasörüne kaydediyoruz.
import os
os.makedirs('../models', exist_ok=True) # Klasör yoksa oluştur
joblib.dump(final_model, '../models/maintenance_model.pkl')

print("\n Model başarıyla kaydedildi: models/maintenance_model.pkl")

Final Model Eğitiliyor...
Model Eğitimi Tamamlandı.

--- Final Model Performansı ---
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1932
           1       0.70      0.85      0.77        68

    accuracy                           0.98      2000
   macro avg       0.85      0.92      0.88      2000
weighted avg       0.98      0.98      0.98      2000


 Model başarıyla kaydedildi: models/maintenance_model.pkl


## Final Pipeline ve Model Mimarisi Raporu

Bu notebook, geliştirilen makine öğrenmesi projesinin **üretim hattıdır (Pipeline).**

### 1. Veri İşleme Stratejisi
Veri sızıntısını önlemek ve modelin genelleme yeteneğini artırmak için şu adımlar standartlaştırılmıştır:
* **Encoding:** Kategorik `Type` değişkeni Ordinal Encoding (L=0, M=1, H=2) ile sayısallaştırılmıştır.
* **Feature Engineering:** Fiziksel tutarlılığı sağlamak adına `Power_W`, `Temp_Diff` ve `Strain` özellikleri, ham sensör verilerinden türetilerek modele entegre edilmiştir.
* **Temizlik:** `UDI`, `Product ID` gibi kimlik bilgileri ve `TWF`, `HDF` gibi hedef sızıntısı yaratan değişkenler pipeline'dan çıkarılmıştır.

### 2. Model Seçimi ve Nedeni
Yapılan kapsamlı testler (Baseline vs Advanced) ve optimizasyonlar sonucunda **Random Forest Classifier** algoritmasında karar kılınmıştır.
* **Neden Random Forest?**
    * Lineer olmayan ilişkileri (Hız-Tork eğrisi) mükemmel modeller.
    * Gürültüye karşı dirençlidir (Overfitting riski düşüktür).
    * `n_estimators=200` parametresi ile en kararlı (stable) sonuçları vermiştir.

### 3. Çıktı (Artifacts)
Eğitilen final model, `maintenance_model.pkl` adıyla serileştirilerek (serialized) kaydedilmiştir. Bu dosya, Streamlit web uygulamasında canlı tahmin üretmek için kullanılacaktır.